<a href="https://colab.research.google.com/github/rjz46/toxic-interpretability/blob/main/Model_Interpretability_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Installations

In [3]:
!pip install seaborn transformers matplotlib pandas captum lime datasets

     |████████████████████████████████| 2.3MB 13.9MB/s 
     |████████████████████████████████| 4.4MB 33.3MB/s 
     |████████████████████████████████| 276kB 30.5MB/s 
     |████████████████████████████████| 225kB 46.2MB/s 
     |████████████████████████████████| 3.3MB 33.7MB/s 
     |████████████████████████████████| 901kB 43.0MB/s 
     |████████████████████████████████| 112kB 59.5MB/s 
     |████████████████████████████████| 245kB 54.6MB/s 
  Created wheel for lime: filename=lime-0.2.0.1-cp37-none-any.whl size=283846 sha256=75c0272c5dbbee17f8f660b8cdcc8ba5b7b1be1f4783df4d680c4209c3a1a407
  Stored in directory: /root/.cache/pip/wheels/4c/4f/a5/0bc765457bd41378bf3ce8d17d7495369d6e7ca3b712c60c89
Successfully built lime


# Imports

In [4]:
#standards
import numpy as np
import scipy as sp
import pandas as pd

#systems
import sklearn
import random
import os
import re
import logging
import sys
from tqdm import tqdm
import collections

#visuals
import seaborn as sns

import matplotlib.pyplot as plt; plt.rcdefaults()
import matplotlib.pyplot as plt
from IPython.display import HTML, display

#sklearn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn import linear_model
from sklearn import neighbors

from IPython.display import HTML, display

#transformers
from transformers import BertTokenizer, AutoTokenizer, BertForSequenceClassification, AdamW, BertConfig

#lime
import lime
from lime.lime_text import LimeTextExplainer

#integrated gradients
from captum.attr import visualization as viz
from captum.attr import IntegratedGradients, LayerConductance, LayerIntegratedGradients
from captum.attr import configure_interpretable_embedding_layer, remove_interpretable_embedding_layer

#pytorch
import torch
import torch.nn as nn


# Load/Process Data

In [5]:
#Load/Parse data
from datasets import load_dataset

#hatexplain dataset
dataset_hate = load_dataset('hatexplain')

#movies dataset
dataset_sentiment = load_dataset('movie_rationales')

Dataset hatexplain downloaded and prepared to /root/.cache/huggingface/datasets/hatexplain/plain_text/1.0.0/df474d8d8667d89ef30649bf66e9c856ad8305bef4bc147e8e31cbdf1b8e0249. Subsequent calls will reuse this data.


Using custom data configuration default


Dataset movie_rationales downloaded and prepared to /root/.cache/huggingface/datasets/movie_rationales/default/0.1.0/250d4f1e030601f5eaec25ace4fc61ef6cf38508d9a761a72cc7142d3207352d. Subsequent calls will reuse this data.


In [6]:
#print(dataset_hate)
#print(dataset_sentiment)

#print(dataset_hate['train'])
#print(dataset_sentiment['train'])

def investigate_data(dataset):

  df = pd.DataFrame.from_dict(dataset['train'])

  #df = pd.DataFrame({"annotators": dataset_hate["train"]["annotators"],
  #                   "post_tokens": dataset_hate["train"]["post_tokens"]})

  pd.options.display.max_colwidth = 0
  display(df.head())

investigate_data(dataset_hate)
investigate_data(dataset_sentiment)


,annotators,id,post_tokens,rationales
0,"{'annotator_id': [203, 204, 233], 'label': [0, 2, 2], 'target': [['Hindu', 'Islam'], ['Hindu', 'Islam'], ['Hindu', 'Islam', 'Other']]}",23107796_gab,"[u, really, think, i, would, not, have, been, raped, by, feral, hindu, or, muslim, back, in, india, or, bangladesh, and, a, neo, nazi, would, rape, me, as, well, just, to, see, me, cry]","[[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]"
1,"{'annotator_id': [27, 6, 4], 'label': [2, 2, 0], 'target': [['Minority', 'Refugee', 'Indian'], ['Indian'], ['Refugee', 'Islam']]}",9995600_gab,"[the, uk, has, threatened, to, return, radioactive, waste, to, the, eu, if, an, agreement, cannot, be, reached, can, not, we, keep, the, radioactive, waste, and, send, back, all, the, paki, migrants, instead]","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1]]"
2,"{'annotator_id': [209, 203, 222], 'label': [2, 2, 2], 'target': [['Other'], ['Hindu'], ['Hindu', 'Other']]}",1227920812235051008_twitter,"[if, english, is, not, imposition, then, hindi, is, also, not, imposition, shut, up, chutiya, retards, stophindiimposition]","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0]]"
3,"{'annotator_id': [235, 222, 209], 'label': [2, 2, 2], 'target': [['Hindu'], ['Hindu'], ['Hindu']]}",1204931715778543624_twitter,"[no, liberal, congratulated, hindu, refugees, post, cab, because, they, hate, hindus]","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1], [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1]]"
4,"{'annotator_id': [51, 25, 4], 'label': [2, 2, 2], 'target': [['Economic'], ['Economic'], ['Caucasian']]}",1179102559241244672_twitter,"[he, said, bro, even, your, texts, sound, redneck, whatever, the, fuck, that, means, 😂, 🤷, 🏻, ‍, ♂️, <user>]","[[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]"


evidences  ...                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  

In [47]:
train_hate, test_hate, validation_hate = datasets.dataset_hate.split(annotators, post_tokens)

NameError: ignored



# Model - BERT

In [7]:

from transformers import AutoTokenizer, BertTokenizer

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print(device)

tokenizer = AutoTokenizer.from_pretrained('bert-base-cased')

model = BertForSequenceClassification.from_pretrained(
    "bert-base-uncased", # Use the 12-layer BERT model, with an uncased vocab.
    num_labels = 2, # The number of output labels--2 for binary classification.
                    # You can increase this for multi-class tasks.   
    output_attentions = False, # Whether the model returns attentions weights.
    output_hidden_states = False, # Whether the model returns all hidden-states.
)

optimizer = AdamW(model.parameters(),
                  lr = 2e-5, # args.learning_rate - default is 5e-5, our notebook had 2e-5
                  eps = 1e-8 # args.adam_epsilon  - default is 1e-8.
                )


model.cuda()



cuda


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [8]:
import torch

# If there's a GPU available...
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla V100-SXM2-16GB


In [ ]:
import time
import datetime

def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))
    
    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [ ]:
import os

# Saving best-practices: if you use defaults names for the model, you can reload it using from_pretrained()

output_dir = './model_save/'

# Create output directory if needed
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

print("Saving model to %s" % output_dir)

# Save a trained model, configuration and tokenizer using `save_pretrained()`.
# They can then be reloaded using `from_pretrained()`
model_to_save = model.module if hasattr(model, 'module') else model  # Take care of distributed/parallel training
model_to_save.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)

# Good practice: save your training arguments together with the trained model
# torch.save(args, os.path.join(output_dir, 'training_args.bin'))



In [ ]:
# Mount Google Drive to this Notebook instance.
from google.colab import drive
    drive.mount('/content/drive')

# Copy the model files to a directory in your Google Drive.
!cp -r ./model_save/ "./drive/Shared drives/ChrisMcCormick.AI/Blog Posts/BERT Fine-Tuning/"

# Load a trained model and vocabulary that you have fine-tuned
model = model_class.from_pretrained(output_dir)
tokenizer = tokenizer_class.from_pretrained(output_dir)

# Copy the model to the GPU.
model.to(device)

# Integrated Gradients

# Attention

# Erasure

In [ ]:
#LIME




In [23]:
#Erasure




In [ ]:
#Integrated Gradient




In [ ]:
#Attention




In [7]:
def predict(inputs, token_type_ids=None, position_ids=None, attention_mask=None):
    return model(inputs, token_type_ids=token_type_ids,
                 position_ids=position_ids, attention_mask=attention_mask, )
    




# LIME

# Evaluation

In [ ]:
#Evaluations

In [48]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Analysis

In [44]:
# Evaluations

In [37]:
#types of words test
spacy

failures
#systematic failure classmethod

#kl_divergence_test
def kl_divergence(datsaet, set1, set2):

#overlap_test

test

NameError: ignored